# 東森新聞雲爬蟲練習
## 練習從東森新聞雲網站中，爬取新聞摘要及詳細內容。
## 學習利用Selenium模擬人為操作，更新動態網頁後爬取新聞內容。

In [12]:
import requests
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

### 以財經新聞為例，先連結到財經新聞網頁，複製其URL。

In [13]:
ETtoday_url = "https://www.ettoday.net/news/focus/%E8%B2%A1%E7%B6%93/"  #財經新聞

### 用Selenium打開一個瀏覽器，連結到網站。然後模擬下拉網頁的動作，讓網頁完成更新，再處理後續動作。

In [14]:
browser = webdriver.Chrome(executable_path='chromedriver/chromedriver')
browser.get(ETtoday_url)  # 打開瀏覽器並連到東森新聞雲網頁

SCROLL_PAUSE_TIME = 1

#
# 以下是用Selenium模擬下拉網頁動作，讓網頁更新
#
last_height = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

while True:
    # Scroll down to the bottom.
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    
    # Wait to load the page.
    time.sleep(2)
    
    # Calculate new scroll height and compare with last scroll height.
    new_height = browser.execute_script("return document.body.scrollHeight")

    if new_height == last_height:
            break
    last_height = new_height

### 到這裡網頁已經更新完畢，所有的新聞都已經出現在網頁上。接下來做爬取收集新聞的動作。

In [17]:
#
# 這裡先建立一個函數，其功能是連到外部連結，並爬取新聞內容。
#
def getNewsDetailContent(link_url):
    resp = requests.get(link_url)
    resp.encoding = 'utf-8'
    #print(resp.text)

    soup = BeautifulSoup(resp.text, 'lxml')
    news_content = soup.find("div", attrs={'class':'story'}).find_all("p")
    for p in news_content:
        """
        .string屬性說明：
        (1) 若當前tag節點底下沒有其他tag子節點，會直接抓取內容(返回"NavigableString")
        (2) 若當前tag節點底下只有唯一的一個tag子節點，也會直接抓取tag子節點的內容(返回"NavigableString")
        (3) 但若當前tag節點底下還有很多個tag子節點，.string就無法判斷，(返回"None")
        """
        if ((p.string) is not None):
            print(p.string)

### 解析HTML並萃取新聞摘要，若有外部的連結，再連到外部連結並把詳細新聞內容抓取下來。

In [18]:
# 爬取網頁內容，解析後萃取新聞摘要
html = browser.page_source
soup = BeautifulSoup(html, "lxml")
all_news = soup.find("div", attrs={'class':'block block_1 infinite_scroll'})

news_block = all_news.find_all('div', attrs={'class':'piece clearfix'})

for i, news_item in enumerate(news_block):
    print("----------------------------------------------------------------------")
    news_body = news_item.find('h3')
    print("\n[%d] %s\n" % (i, news_body.a.string))
    
    # 
    # 連到外部連結，擷取詳細新聞內容
    #
    externalLink = "https://www.ettoday.net" + news_body.a["href"]
    getNewsDetailContent(externalLink)


----------------------------------------------------------------------

[0] 外資狂賣！大同股價重摔　市場派提聲明呼籲政府「拒絕違法董事會、保障股東權益」

▲大同近期股價低迷。（圖／記者湯興漢攝）
記者陳心怡／台北報導
大同因身陷經營權之爭，近期股價低迷，昨天跟今天都跌停板，外資更是連續9個交易日賣超大同，針對大同股價跌不停，市場派羅得等3家投資公司今發表聲明，請政府做主「拒絕違法董事會、保障股東權益」，強調大同6月30日股東會之亂，股東卻無法獲准自行召開股臨會，股價跌不停，公司違法卻是全體股東受害。
羅得、三雅和競殿公司以大同股東身分指出，公司派在今年股東會違法亂紀，大同因此被證交所打入全額交割股，MSCI於8月中決定剔除大同成分股，昨天開盤大同就跌停鎖死，收盤還委掛8765張賣出，經濟部不准3%股東開股臨會，無法終結大同之亂，讓全體股東承受惡果。
羅得等投資公司今天在報紙刊登半版廣告，發表5點聲明：
1. 遲來的正義不是正義！有為的政府機關可以依法直接准許3%股東召集開會，不必留待司法訴訟！
3. 經濟部如要求股東依公司法第173條第1項規定，請求109年6月30日「違法就任而無效」之董事會召集臨時股東會，無異是「請鬼提藥單」，將「違法」董事會「變相」承認為「合法」？顯然是行政權之自我投降與退讓，唯有透過公司法第173條第4項之規定，使3%股東自行召集股東會，才是解決本件爭議的根本辦法。
4. 我等立場如同主管機關，堅定認為大同公司109年6月30日股東會剔除53.32%股東權為違法不成立、無效，不願配合大同公司重演上一屆「靠訴訟拖過任期」的歹戲！
5. 請經濟部勇於擔當，直接准由3%股東召開股東會，就可合法終結大同公司109年6月30日違法股東會的亂象！
----------------------------------------------------------------------

[1] 感念李登輝率先開啟南向大門　經長王美花：為台灣產業走向國際奠基

▲經濟部長王美花回應時事。（圖／記者林敬旻攝）
記者林淑慧／台北報導
李登輝前總統昨（30）日晚間辭世，享耆壽九十八歲，經濟部長王美花今日表示，李前總統於九○年代啟動南向政策，鎖定菲律賓、泰國、馬來西亞、印尼、新加坡、越

▲上市品牌建商櫻花建設搶進北屯機捷特區，推出低單低總宅「櫻花巨人」，幫助年輕首購族輕鬆成家。（採訪撰稿／記者鍾欣妍；攝影剪輯／記者謝婷婷）
記者鍾欣妍／採訪報導
台中第一條捷運綠線將在2020年底通車，其中扮演捷運首站的北屯區，還有美式賣場Costco二店要拚年底完工營運，讓北屯機捷特區成為首購熱區。
▲▼台中市不動產代銷經紀商業同業公會理事長林傳傑表示，北屯機捷特區位居台中捷運綠線首站，繼美式賣場Costco二店進駐加持，帶動北屯區房市。（圖／記者謝婷婷攝）
台中市不動產代銷經紀商業同業公會理事長林傳傑表示，受惠於機捷特區和Costco將進駐，看屋人潮每週平均上看80組，北屯區房市後勢仍看漲，「在這個區塊，房價每坪24萬到28萬都有，後續上看3字頭，對於這個區塊未來的房價，我是建議年輕人要買房，我覺得應該要及時趁早。」
▲▼位在北屯機捷特區的「櫻花巨人」，產品規劃26到59坪。（圖／記者謝婷婷攝）
深知年輕人買房肩上負擔沈重，上市品牌建商櫻花建設搶進北屯機捷特區，推出預售案「櫻花巨人」，產品規劃26到59坪，搭配低單、低總，還加碼祭出輕鬆付款方案。
▲櫻花巨人專案副總何世宰表示，本案推出低單低總、結構工程零付款等方案，幫助年輕首購族買房。（圖／記者謝婷婷攝）
櫻花巨人專案副總何世宰說：「在交屋的時候，公司會提供購屋者5年的Costco免費的會員卡，在付款方面，有一個結構工程零付款，他買的時候只要付簽約金之後，其他的部分在房子蓋到好之後，門牌取得的時候，再來繳10%而已，其他都是8成貸款。」
▲▼「櫻花巨人」擁雙捷運雙鐵共構，基地位置鄰近松竹、頭家厝、軍功商圈和優質學區。（圖／記者謝婷婷攝）
擁有1800坪壯闊基地和三面臨路角地優勢，附近有雙捷運雙鐵共構，距離松竹、頭家厝和軍功商圈則是10分鐘車程，交通和機能兼具，基地位置也鄰近優質學區。
▲▼公設規劃有健身房（上圖）、電影院（下圖）、兒童遊戲區等豐富設施。（圖／櫻花巨人提供）
何世宰強調：「鄰近台中市的明星學校，葳格中學、華盛頓國小，還有東光國小，還有北新國中，所以這裡不論是公立學校或是私立學校都相當方便，大概都在五分鐘車程內就會到，本案有規劃健身房、兒童遊戲室、電影院，還有閱讀室，讓你平常下班後，不用到別的健身房，在社區裡就可以健身了。」櫻花建設多年採用薄利多銷模式銷售，這次借出巨人的肩膀，要幫助年輕首購族輕鬆實

▲台達電執行長鄭平。（圖／記者楊絡懸翻攝）
記者楊絡懸／台北報導
台達電（2308）30日舉行法人說明會。執行長鄭平表示，第三季電源零組件優於ICT（資通訊）表現，IA（工業自動化）的事業表現也還可以，預估第三季營收將比第二季好；至於第四季則還不清楚，仍有很多未知數，盼毛利率維持30%以上。
新冠肺炎（COVID-19）疫情衝擊產業狀況，台達電董事長海英俊表示，原先預期台達電的自動化設備也會受影響，卻受惠於口罩、呼吸器的急單帶動，因此台達電上半年仍維持成長，可望下半年因工廠自動化趨勢有雙位數成長。
海英俊說，5G受疫情影響，發展進度比預估慢，但最近網路需求漸升、各國5G推動也變得較積極，因此帶動如5G手機相關產品成長，被動元件的稼動率滿載，利潤也高，尤其下半年多數品牌5G手機將上市，屆時將利用到大量的被動元件。
在電動車（EV）方面，車市衝擊台達電EV電源。海英俊透露，電動車是趨勢，台達電手上訂單很多，卻因疫情致汽車組裝廠於上半年停工，出貨放緩；目前各工廠陸續復工交貨，狀況漸好轉，若訂單如期出貨，營收貢獻將表現不錯。
▲台達電董事長海英俊。（圖／記者楊絡懸翻攝）
法人關心台達電印度新廠進度，原先今年3月就要完成，卻受疫情影響產能布局，海英俊指出，台灣目前有桃園五廠、南科三廠等新產線，若整理平鎮廠區，也有產能空間可以規劃。
至於市場關心「紅色供應鏈」逐漸逼近而色變、是否將影響台廠規劃？海英俊說，這問題不是今天才有，任何一個行業都有很多競爭對手，而中國廠商「強的地方」還是靠成本。
海英俊進一步解釋，對抗紅色供應鏈，絕對不是拚低成本，因為永遠都有更低的成本；若要擺脫紅海，就要在產品技術與想法上「增加價值」，就如台達電早就決定一定要在技術上領先，無論是在2008年金融海嘯或是2020年的新冠肺炎等環境動盪，台達電研發的費用與力道並沒有因此減弱，甚至還加強，保持更好的技術品質優勢。
台達電第二季受惠於筆電、伺服器等相關零組件需求強勁，合併營收706.94億元，季增27%、年減3.7%；毛利率32.9%，季增6.4個百分點、年增5.88個百分點；營益率13.1%，季增9.5個百分點、年增5.79個百分點；稅後淨利75.97億元，季增268%、年減20%；每股純益（EPS）2.92元。
累計上半年營收1263.73億元，年減0.9%、毛利率30.1%；稅後純益96.65億元，年減

▲坐落於員林市浮圳路二段與育英路交會處工業區土地，地點相當方便。（圖／台灣金聯提供）
記者吳靜君／台北報導
因應台商資金回台、投資需求，台灣金聯釋出三筆不動產，包含台北市內湖、苗栗銅鑼及彰化員林等千坪到萬瓶可供建廠的大面積房舍或建築用地，預計8月13日開標。
台灣金聯董事長施俊吉表示，這次的第一標，是在台北市內湖區的1,356.1坪的大型商用地下室；距離內湖線葫洲站僅240公尺，內含一坡道車位有面積超過五公尺的獨立出入口，適合大型超市、量販店與藝術展覽館、辦公室使用，地點頗佳。
第二標事位在苗栗縣西湖鄉的工業建築素地，面積1萬460.47坪，施俊吉表示，該土地容積率為300%、且地勢平坦已經有多組買家詢問；但是該土地原地主有遺棄事業廢棄物，基於社會責任，金聯已經投入數千萬元整治，經三年整治後去（2019）年的8月29日苗栗縣政府公告解除土壤汙染控制場址。
第三標則是位在員林市浮圳路二段與育英路交會處3060.58坪工業區土地；該土地對側為即為華成市場，位處精華地區，地形方正且相當完整，可獨立開發成商辦、賣場、超市等建築，而土地分區深富變更使用之彈性，所以目前標脫可能性最高。
----------------------------------------------------------------------

[34] 富達投信調查：台灣職業女性愛投資、重視退休理財　但決策信心不足

▲調查發現台灣職業女性對自己的投資決策信心不足。（圖／取自免費圖庫pixabay）
記者陳心怡／台北報導
富達投信首次將大規模女性投資主題調查帶來台灣，發佈「投資成就新女力」調查報告。調查發現84%的台灣職業女性有投資行為，相對日本（23.5%）和澳洲（48%）更高。調查也顯示，台灣女性在個人財務管理上擁有自主權，但是她們對自己的投資決策信心不足；此外，台灣女性高度重視退休儲蓄，但財務計劃缺口較大。
富達投信董事長陳思伊表示:「作為投資與退休領域的專家，我們希望針對女性理財投資與退 休規劃，詳細分析台灣女性在投資理財過程中所遇到的困難，探討解決之道及提出建議，幫助 更多台灣女性通過投資達到個人目標，成就更美好的人生。」
這次調查發現，台灣職業女性的平均總流動資產為新台幣 250 萬元，佔台灣男性平均的 61% (新台幣 410 萬元)。雖然台灣女性相對缺乏資本，但有八成台灣職業女

▲第一金暨一銀董事長廖燦昌。（圖／記者吳靜君攝）
記者吳靜君／台北報導
第一金、一銀董事長廖燦昌因為涉入「遠航案」遭到起訴；廖燦昌今（30）日也遞交辭任書財政部也核准，財政部表示，廖燦昌的第一金董座與一銀董座的職位則由第一金總經理林謙浩與一銀總經理鄭美玲暫代三個月。
財政部表示，公股代表第一金董事長廖燦昌、土銀董事長黃伯川以及合庫總經理陳世卿因為涉嫌遠航的貸款，台北地檢署遭到起訴。
廖燦昌請辭金控董事長以及其他子公司兼職，財政部予以准辭；至於黃伯川、陳世卿兩人的部分，財政部要進一步了解案情，未來事證做妥適處理。
財政部今日表示，至於廖燦昌留下的第一金董事長的職缺則由林謙浩暫代、一銀董事長之缺則由一銀總經理代理；三個月內財政部要提新的董座人選；如果三個月內沒有新人選，就要報到行政院。財政部表示，廖燦昌、黃伯川是首次在董事長任內遭到檢方起訴的董事長。
▼一銀董事長廖燦昌（左）、一銀總經理鄭美玲（右）。（圖／記者吳靜君攝）
----------------------------------------------------------------------

[44] 理財周刊／後12682　優勝劣敗淘汰賽

文／洪寶山
美東時間7月24日COMEX黃金睽違11年首度站上1900美元，今年以來上漲24.77%，從2018年開啟美中貿易戰以來，COMEX黃金累積45.6%漲幅，自2月28日見刊(1018期)的「全球理財觀」專欄就白紙黑字的提示到：「市場對於這次疫情的擔憂情緒增溫，避險需求增加，當然還有對美國與中國兩大央行的降息預期，醞釀了金價有機會挑戰1800美元關卡，甚至二千美元的整數關卡。十年一次的大行情，當然不能眼睜睜的溜走。」
COMEX黃金站上1900美元關卡
既然是十年一次的大行情，本欄就時不時的耳提面命的提示，幾乎是每個月的追蹤COMEX黃金的最新觀點，相信只要是本欄的忠實讀者，這場天災人禍(COVID-19疫情與中美貿易戰)百年難得一遇亂世，千載難逢的「金」牛，利用元大S&P黃金ETF(00635U)，您一定是賺得盆滿缽滿，開心的笑不攏嘴，面對7月28、29日的聯準會利率決策會議，截至7月27日金價已突破2011年1921美元的歷史高點，接下來是先回落整理，還是二話不說，直接油門踩到底，直衝二千美元大關？
避險資金「棄美元、就黃金」
關鍵就在美元指數。

在台積電、聯發科聯袂大漲中，台股終於創新一三○三一點歷史新高，且上市櫃指數出現歷史最大量，雖然指數放大量後屢屢收黑，不過國際資金依舊澎湃！台積電供應鏈養分十足，配合聯發科攻占４Ｇ、５Ｇ領導地位，以台灣半導體全球地位，美中同時要拉攏，趨勢向上不變；蘋果新機預計兩個月後將登場，相關供應鏈業績也看好；同時國內壽險公司持續大筆資金買地，間接提升資產價值。台股大量後是有拉回壓力，惟帶領台股攻堅的族群基本面佳，與三○年前炒高至歷史高點後崩跌內涵不同，好股逢低仍可尋找買點。
文／方亞申
過去一周，台股受到英特爾七奈米製程出現瑕疵，將比原訂時程延後半年推出，台積電股價從三八六元一度大漲至四六六．五元，兩天內漲幅超過二成。英特爾過去在十奈米也出現這種狀況，等於宣告技術追不上台積電。過去三○年，英特爾一直是全球晶片製造龍頭，相較美國多數晶片業者關廠，或是賣掉當地工廠、將產線移往亞洲，英特爾始終未跟進，仍堅守在美營運，期待能做出更棒的產品。如今英特爾先進製程十奈米及五奈米頻頻卡關，先進製程延後半年推出，若決定委外代工，最有可能交給台積電生產。
先前台積電法說會不僅公布第二季獲利及毛利率創歷史新高，也增加資本支出十億美元，達到一六○至一七○億美元，主要在於五奈米製程投產、三奈米製程一年後推出、二奈米加緊研發，加上先進封裝技術，提高金額超乎市場預期，如今英特爾又宣告技術突破還需要時間，此現象暗示著五年內英特爾可能都追不上台積電，於是大幅提升台積電評價目標區，最高至五五○元。
站在十日線之上都屬強勢
以台積電上半年ＥＰＳ九．一七元創新高，且七奈米製程出貨占第二季晶圓銷售金額的三六％；十六奈米製程出貨占全季晶圓銷售金額的十八％。總體而言，先進製程的營收達到全季晶圓銷售金額的五四％，刺激毛利率上升。市場預估後續英特爾明年將吃下十八萬片台積電六奈米製程，等於保障台積電基本獲利。
見諸全球晶圓代工高階製程（十六奈米以下），台積電幾乎通吃，接下來三奈米及二奈米已推出時間表，全球領先地位穩固，中國雖有意藉由中芯半導體追趕，但至少差了三個世代以上、三星也差了一個世代以上，預計未來兩年台積電依舊是最先進製程者。而今年外資預估台積電ＥＰＳ將從十九．五元附近起跳，明後年將達二二至二四元。若用本益比（ＰＥ）及股價四三○元換算，今年ＰＥ約二二倍、明年約二○倍、後年約十七．九倍。以台積電二五九三．○四億元台股最大股本，

記者曾筠淇／綜合報導
▲許多店家都傾向綁定信用卡，進而讓民眾消費越來越不知不覺。（示意圖／記者紀佳妘攝）
Elaine理白小姐同時也提供了3種減少被動支出的方法：
Elaine理白小姐舉例，由於KKbox可以共享，因此曾與朋友一起使用，而她在檢視信用卡帳單後才發現，KKbox已自動扣款月費，導致自己雙重付款。由於許多商店都傾向綁定信用卡，進而讓民眾的消費越來越被動、沒感覺，因此一定要定期檢視自己的帳單，或是以記帳的方式來檢查自己的支出是否有必要，減少自己的支出。
----------------------------------------------------------------------

[60] 快訊／光寶科啟動「二代接班」！宋恭源長子宋明峰接董事長　副董陳廣中也卸任交棒

▲光寶科技董事會宣布啟動二代接班！圖為光寶科技董事長宋明峰、光寶集團董事長宋恭源、光寶集團副董事長陳廣中、光寶科技總經理邱森彬。（圖／記者楊絡懸攝）
記者楊絡懸、王曉敏／台北報導
光寶科技（2301）董事會30日宣布啟動二代接班！由董事長宋恭源長子、光寶科技智能生活與應用事業群執行長宋明峰，接任董事長，即日起生效；同時，光寶科技總經理由原光寶科技電能事業群執行長邱森彬接任，自8月1日起生效。
光寶科表示，宋明峰接任董事長，未來將著重於公司治理及董事會運作、核心人才培育與長期成長策略，持續鞏固光寶的核心競爭力與企業文化傳承；前董事長宋恭源卸任後，仍留任董事會及集團董事長。
此外，光寶科技副董事長暨總執行長陳廣中也將卸任，仍為光寶科董事、集團副董事長，將持續提供諮商於集團永續經營及發展、產官學公會關係、併購、數位轉型與智能製造等方面。
宋恭源表示，光寶董事會感謝並高度肯定陳廣中37年全心投入光寶，於總執行長任內完成集團資源整合並致力於精實聚焦，優化光寶營運體質；此人事令除了年輕化經營梯隊，也期望借重資深主管豐富的產業經驗，攜手傳承光寶優良的營運實績及企業文化，實踐「百年企業」的願景。
陳廣中則笑說，自己進入光寶已37年，其中25年獨當一面，他執行精實聚焦、深度整合的工作，完成光寶集團的階段性任務，如今他也71歲了，家人與子女都遠在美國，他長年分隔兩地、專注於工作，對家人深感虧欠，因此退休後將會到美國陪伴家人，用有限的時間補償家人。
-------------------------

▲慧智基因總經理洪加政。（圖／記者姚惠茹攝）
記者姚惠茹／台北報導
慧智基因（6615）今（30）日宣布，為擴大泰國服務規模，今年5月將泰國子公司遷至曼谷醫療精華區，除了規劃在泰國即可進行檢體檢測的實驗室外，更放眼東南亞龐大的醫療市場，目標將泰國子公司打造為東南亞基因檢測中心。
慧智基因表示，看準東南亞龐大的基因檢測市場，以及泰國先進的醫療水準和優越的地理位置，2019年初在泰國曼谷成立子公司，並建立基因檢測實驗室，隨著基因檢測業務量日益成長，慧智基因今年五月將泰國子公司遷至曼谷的Sukhumvit，持續擴大服務規模。
慧智基因總經理洪加政表示，慧智基因泰國子公司設置的Sukhumvit不僅是曼谷金融重鎮，也鄰近多家醫學中心與婦幼醫院診所，例如東南亞最先進醫院集團之一的Samitivej醫院、曼谷醫院及Bumrungrad國際醫院等，是泰國醫療體系的精華區。
慧智基因指出，泰國為全球知名的醫療旅遊目的地，近幾年醫療市場成長快速，慧智基因設立泰國子公司至今，除了希望能夠移植台灣多元的基因檢測項目外，也希望能夠推出在地化的基因檢測服務，與台灣產生互補效應。
▲慧智基因泰國子公司遷至曼谷醫療精華區。（圖／記者姚惠茹攝）
慧智基因說明，目前慧智基因在泰國主要的服務項目為孕前檢測、癌症基因檢測及新生兒檢測，也在規劃胚胎著床前基因診斷(PGD)、胚胎著床前染色體篩檢(PGS)等服務，而且泰國擁有多間獲得國際醫院評鑑(JCI)的醫院，醫療品質位列東協第一，期望與當地頂級醫療機構合作。
洪加政指出，慧智基因不僅要推出在地化的基因檢測服務外，也規劃打造在泰國即可進行檢體檢測的實驗室，讓泰國子公司可以自行完成一定比例的基因檢測項目，增加基因檢測的效率，進而推動基因檢測服務擴散，朝向東南亞各國發展。
慧智基因強調，泰國政府近年來積極推動醫療產業發展，致力將泰國打造為「亞洲醫療中心」。而根據工研院IEK Consulting報告預估，東協與南亞2022年醫療支出總和將達到4,439億美元，成長速度為全球區域市場之最。
洪加政強調，慧智基因看好東南亞醫療市場發展的潛力，未來規劃以泰國為中心，朝東南亞潛在市場像是寮國、柬埔寨、新加坡、馬來西亞、越南、印尼等國發展，下一步的目標是將泰國子公司提升為東南亞基因檢測服務的中轉站，提供東南亞各國完善的基因檢測服務。
------------------

▲左起為中天生技藥物研究所副所長柯逢年、合一生技董事長黃山內、中天生技財務副總經理鄭志慧（圖／翻攝櫃買）
記者姚惠茹／台北報導　
中天生技（4128）旗下合一（4743）開發ON101新藥三期解盲，利多消息帶動今（30）日股價開盤亮燈漲停，並激勵生技指標股「天國一輝」的杏輝（1734）、杏國（4192）同樣攻上漲停，但台新投顧副總黃文清表示，近期波動太大，建議投資人應該要回歸基本面跟實質的獲利來操作。
合一昨（29）日晚間公告，自行開發的糖尿病足慢性傷口潰瘍（DFU）新藥ON101，完成三期臨床試驗解盲，主要療效指標16周傷口完全癒合率達到62.2％，是目前全球唯一三期試驗超過六成完全癒合率的新藥。
利多消息激勵生技指標股「天國一輝」齊攻漲停，其中合一開盤跳空漲停來到168.5元，截至10:30委買高掛6077張，而中天生技同樣跳空漲停來到66.8元，委買高掛7342張，至於杏輝則是盤中攻上漲停來到38.95元、杏國一樣開盤漲停來到65.7元。
黃文清表示，生技股面臨到季報要公布的時間點，投資人會開始檢驗大漲之後，生技股到底有沒有實質的獲利，所以會偏向整理的格局，尤其是近期波動太大，建議投資人應該要回歸基本面跟實質的獲利來操作，並特別小心投資的風險，更推薦操作進入旺季的科技類股。
「天國一輝」近期股價波動劇烈，其中像是合一連連漲停多日之後，強攻生技股王寶座，最高來到476.5元，但不到幾天便開始一路跌停至昨日收盤價153.5元，跌幅高達67.8％，短線跌幅太深，但隨著新藥三期解盲的利多消息，帶動生技類股走向反彈格局。
----------------------------------------------------------------------

[83] 李沃牆／Fed維持利率近零　美元長期走貶不利競爭力

▲美國聯準會宣布將聯邦基金利率維持在0%-0.25%目標內。（圖／路透）
●李沃牆／淡江大學財金系教授
美國聯準會（Fed）於28、29日舉行為期兩天的利率決策會議，並於台北時間（30日）凌晨發表會議聲明。一如市場預期，Fed將聯邦基金利率（federal funds rate）維持在0%-0.25%的目標內，直到步上全面就業和物價穩定的目標；並表示新冠肺炎疫情帶來相當大風險，將動用所有工具支持搖擺不定的美國經濟。
究竟，美國經濟能否起死回升，步向

▲保德信投信總經理張一明。（圖／記者湯興漢攝）
文／陳心怡
許多人從來沒想過，台股居然會在疫情還沒真正結束前，創下歷史新高點。保德信投信總經理張一明接受《ETtoday新聞雲》專訪時指出，大家都知道現在市場是用金錢堆積起來的，很多市場估值容易受風吹草動影響，造成回檔的藉口，所以在這種表面上看起來樂觀，但其實充滿回檔風險的情況下，不斷提醒客戶要注重風險控管，最好的方法就是做資產配置。
張一明認為，展望資產管理產業發展，退休議題是一大關鍵，勞退新制適用的人有八九百萬人，大家提撥的退休金放到大水庫裡面，現在擔心它會破掉，先前有媒體說如果靠著加強投資績效，就不用可慮「出水大於進水」問題，不過他非常反對這種論調，「這種真的不能搏，現在水庫缺的，不是小小5%、10%而已，而是十幾趴，你根本不可能靠著投資去補滿，這問題還是得回歸出水量跟進水量平衡，再來看投資報酬率，所以我覺得下一波會帶動行業發展的，而且是對大家都好，絕對是退休」。
近期，ETF（指數股票型基金）越來越流行，討論度也相當高，張一明認為，有些人會說ETF之所以流行，是因為主動基金不行了，贏不了大盤，所以乾脆買被動操作基金，但是這也說不準，主動操作和被動操作其實是有週期性的，雖然目前大家一面倒向被動操作基金，但其實有很多主動操作基金是持續打敗大盤的，意思是可以創造超額報酬，反觀被動操作，不會有超額報酬，只會有市場報酬而已。
張一明非常有自信地說，保德信投信內部對於基金經理人設定的KPI相當高，不是希望他們的績效只跟大盤一樣，既然跟投資人收取管理費和手續費，一定要創造更好的附加價值，要不然大家都去買ETF就好了，且還比較便宜，因此保德信投信大部分的基金是可以超過目標指數5%以上。
「保德信的文化其實比較保守，因此一方面希望基金經理人可以創造超額報酬，另一方面我們在整個投資流程上控管是相當嚴謹的，包括下檔風險的控管，以及在Tracking error（追蹤誤差）不能太高的情況下，靠著選股打敗大盤，那才是真正的功力。」張一明說。
張一明說，像是「保德信168聰明定期不定額」就是為基金長期投資人提供加強風險控管的服務，投資人既能保有定期定額投資基金的功能，又增加不定額扣款的功能，當基金淨值漲幅達到20%時，系統會幫投資人減少扣款金額，降低投資人買高的風險；當基金淨值跌幅至10%時，系統會幫投資人加碼買進，用更多的資金買進更多價